In [ ]:
import numpy as np
import pickle
import pandas as pd

# G-code generation codes are cloned from https://github.com/tibor-barsi/GcodeGenerator. Tibor Barsi is the author of the code, and was a PhD student at the Ladisk lab of the University of Ljubljana
#  We should be careful with crediting the author of the code, if we ever want to make these code public.
from src.g_code_generation_copy.gcode_generator import G_code_generator
from src.g_code_generation_copy.tool_changer_functions import save_params, load_params, printer_start, load_tool, unload_tool, tool_change, take_photo, play_sound, printer_stop
# from src.additional_functions import *
from src.network import Network_custom, replace_brackets
import os
import plotly.io as pio
pio.renderers.default = 'browser'

Define the unit cell network

In [4]:
def Rot(theta):
    """
    Rotation matrix around the z-axis
    :param theta: angle in radians
    :return: rotation matrix
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])

# Note that the coordinates of the vertices will change after equilibrium is reached.
model_name = "validation_unit"
L_scale = 10.5
vertices    = np.array([[-10, 0, 0], [-5, 0,0], [0, 0, 0], [5, 0, 0], [10, 0, 0], [0,-10,0], [0,-5,0], [0,5,0], [0,10,0]])*L_scale  # The coordinates of the vertices in mm
vertices    = vertices@Rot(np.pi/4)
edges       = [[0, 1], [1, 2], [2, 3], [3, 4], [5, 6], [6, 2], [2, 7], [7, 8], [1, 6], [6, 3], [3, 7], [7,1]]                  # The edges of the network.
paths       = [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11]]   # 3D printable paths. Ensure an edge is not flipped.
directions  = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]          # Flip the curvature of an edge by setting the direction to -1.
center_cross_path = [1, 2, 5, 6]
connector_path = [0, 3, 4, 7]
q_scalar = 1.4
sum_q0 = 0.02*q_scalar
sum_q1 = 0.02*q_scalar
sum_q2 = 0.01*q_scalar
offset = 1/32
ratio_0 = np.array([3/8-offset, 1/8, 1/8, 3/8+offset])
ratio_1 = np.array([3/8, 1/8-offset, 1/8+offset, 3/8])
ratio_2 = np.array([2/8-offset, 2/8+offset, 2/8, 2/8])

q = np.zeros(len(edges))
q[paths[0]] = sum_q0*ratio_0
q[paths[1]] = sum_q1*ratio_1
q[paths[2]] = sum_q2*ratio_2

fixed       = [0, 4, 5, 8]
# Create the network
net = Network_custom.from_fd(vertices, edges, q, fixed, paths = paths, dir = directions)
net.net_plot(color=True, elables = False, vlabels = True)
np.max(net.f)/0.078294515

7.321273822984715

In [5]:
printing_params = load_params(r'DATA/NT_Eel_0.2mm.json')
alpha_loop = np.deg2rad(30) # The angle of the loop
L_loop  = 9                # The length of the loop
n_points = 30               # The number of points in the loop
net.n_split = n_points

start_gcode     = open(r'DATA/start_gcode_XL.gcode', 'r').read()
end_gcode       = open(r'DATA/end_gcode.gcode', 'r').read()

temperature_settings = {'first_layer_bed_temperature': 60, 'first_layer_temperature':220, 'K-factor': 0.20, 'idle_temperature': printing_params["T_nozzle_standby"]}
comment = ''

bed_width = 230
bed_height = 210

start_loop_bools = [True, True, False]
end_loop_bools = [True, True, False]

running_start_bools = [True, True, True]
running_end_bools = [True, True, False]
L_running = 6

file_path = os.path.join('DATA', 'Avg_Stress_Strain_Overture_TPU.csv')
stress_data, strain_data = net.load_stress_strain_curve(file_path, A_scale = 1.0)

TPU_nl = {'stress':strain_data, 'strain': stress_data, 'v':0.3897, 'p':1.18e-9, 'A': 0.078294515, 'name': 'TPU Overture'} # TPU Overture non-conductive

A = [TPU_nl['A']]*len(net.edges)

# Purge line coors
gen = G_code_generator(printing_params)
point0, point1 = [20,15,printing_params['layer_height']-printing_params['first_layer_offset']], [150,15,printing_params['layer_height']-printing_params['first_layer_offset']]
point2, point3 = [150,10,printing_params['layer_height']-printing_params['first_layer_offset']], [20,10,printing_params['layer_height']-printing_params['first_layer_offset']]

def make_purge_line(printing_params, gen, point0, point1, ef_extra = 1.0, comment=''):
    """"
    ""Make a purge line at the start of the print."
    """
    g_code = '\n'
    g_code += gen.move_to_point(point0[0:2], point0[2] + printing_params['nozzle_lift'], comment='Move to start point')
    g_code += gen.move_to_point(point0[0:2], point0[2], comment='Lower Nozzle')
    g_code += gen.unretract()
    g_code += gen._print_line(
            point0=point0,
            point1=point1,
            move_to_start=False, # move to start point without extruding
            extrude_factor=printing_params['extrude_factor']*ef_extra,
            comment=comment)
    g_code += gen.retract()
    g_code += gen.wipe(2 * np.pi)
    g_code += '\n'
    return g_code

def add_net_to_gcode(printing_params, gen, net):
    g_code = '\n'
    for path_i, cor_list in enumerate(net.paths_xyz):
        g_code += '\n;Path (' + str(path_i) + ' ' + str(path_i) + ')\n'
        cor_list   = np.array(cor_list)
        # Move the coordinates to the center of the bed and add the layer height
        cor_list[:,0] += bed_width/2 -14
        cor_list[:,1] += bed_height/2 
        cor_list[:,2] += (printing_params['layer_height'] - printing_params['first_layer_offset'])
        g_code += '\n'
        # Move the coordinates to the start point
        g_code += gen.move_to_point(cor_list[0][0:2], cor_list[0][2] + printing_params['nozzle_lift'], comment='Move to start point')
        # Lower the nozzle
        g_code += gen.move_to_point(cor_list[0][0:2], cor_list[0][2], comment='Lower Nozzle')
        # Unretract the filament
        g_code += gen.unretract()
        # Print the path
        for point0, point1 in zip(cor_list[:-1], cor_list[1:]):
                g_code += gen._print_line(
                point0=point0,
                point1=point1,
                move_to_start=False, # move to start point without extruding
                extrude_factor = printing_params['extrude_factor'],
                comment=comment)
        # Retract the filament
        g_code += gen.retract()
        # Wipe the nozzle
        g_code += gen.wipe_from_last_points(g_code)    
        # Raise the nozzle
        g_code += gen.move_to_point(point1[0:2], point1[2] + printing_params['nozzle_lift'], speed_factor=0.5, comment='Raise Nozzle')
    return g_code

Generate a Specimen

In [11]:
reference_point = [0,0,0] 
l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 1000, "damping": .01, "correction_scalar": 1., "tol": 1e-6})
net.optimize_vertices()
net.scale_vertices(reference_point, net.l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically
R, th = net.arc_param()
xyz = net.arc_points(n = n_points)
net.auto_flip_curves(n = n_points)        # Automatically flip the curvature of the edges. Directions will become 1, -1, 1, -1, ...
net.flip_curves(n = n_points)                   # Flip all the edges.
net.flip_curves([2, 5, 8], n = n_points)
net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 
net.net_plot(plot_type = 'arcs')
net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)
net.add_running_start(running_start_bools, running_end_bools, L_running, n_points)
file_name = model_name + f'_0'

g_code = replace_brackets(start_gcode, temperature_settings)
g_code += make_purge_line(printing_params, gen, point0, point1, comment='Purge line', ef_extra=2.5)
g_code += make_purge_line(printing_params, gen, point2, point3, comment='Purge line', ef_extra=1.05)
g_code += add_net_to_gcode(printing_params, gen, net)

g_code += end_gcode
with open(os.path.join('DATA', 'generated_gcodes', file_name + '.gcode'), "w") as g_code_file:
    g_code_file.write(g_code)

Iteration 0: Current error = 154.71800303957406
Iteration 100: Current error = 48.68434432080412
Iteration 200: Current error = 21.85373738756606
Iteration 300: Current error = 10.166491353648667
Iteration 400: Current error = 4.831744736064693
Iteration 500: Current error = 2.33175922186735
Iteration 600: Current error = 1.138953737349879
Iteration 700: Current error = 0.5627080272919098
Iteration 800: Current error = 0.2818825113872545
Iteration 900: Current error = 0.14418762812926236
Final error: 0.07684537261211453


Generate 6 specimens only changing the tension

In [7]:
q0 = q.copy()
q_scalars = [0.35, 0.6, 0.8,1.0, 1.15, 1.25]
for q_scalar in q_scalars:
    file_name = model_name + f'_fstudy_{q_scalar}'
    q_new = q0*q_scalar
    # net.update_shape(q_new)
    net = Network_custom.from_fd(vertices, edges, q_new, fixed, paths = paths, dir = directions)
    l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
    net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 1000, "damping": .01, "correction_scalar": 1., "tol": 1e-6})
    net.optimize_vertices()
    net.scale_vertices(reference_point, net.l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically
    R, th = net.arc_param()
    xyz = net.arc_points(n = n_points)
    net.auto_flip_curves(n = n_points)        # Automatically flip the curvature of the edges. Directions will become 1, -1, 1, -1, ...
    net.flip_curves(n = n_points)                   # Flip all the edges.
    net.flip_curves([2, 5, 8], n = n_points)
    net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 

    net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)
    net.add_running_start(running_start_bools, running_end_bools, L_running, n_points)
    
    g_code = replace_brackets(start_gcode, temperature_settings)
    g_code += make_purge_line(printing_params, gen, point0, point1, comment='Purge line', ef_extra=2.5)
    g_code += make_purge_line(printing_params, gen, point2, point3, comment='Purge line', ef_extra=1.05)
    g_code += add_net_to_gcode(printing_params, gen, net)

    g_code += end_gcode
    # with open(os.path.join('DATA', 'generated_gcodes', file_name + '.gcode'), "w") as g_code_file:
    #     g_code_file.write(g_code)
    net.net_plot(color=True, elables = False, vlabels = True, plot_type = "arcs")

Iteration 0: Current error = 10.175711153099208
Iteration 100: Current error = 5.001419929972803
Iteration 200: Current error = 2.511056471130024
Iteration 300: Current error = 1.2865296229666718
Iteration 400: Current error = 0.6821968147880202
Iteration 500: Current error = 0.38327534745576597
Iteration 600: Current error = 0.2351749101655186
Iteration 700: Current error = 0.1617112748490615
Iteration 800: Current error = 0.12523620574903957
Iteration 900: Current error = 0.10711028667696504
Final error: 0.09816592740486065


Iteration 0: Current error = 44.9757473492699
Iteration 100: Current error = 21.470780007530788
Iteration 200: Current error = 10.664385679589968
Iteration 300: Current error = 5.414076500440067
Iteration 400: Current error = 2.844394011460173
Iteration 500: Current error = 1.5807231070474461
Iteration 600: Current error = 0.9572379893890296
Iteration 700: Current error = 0.6488685967283778
Iteration 800: Current error = 0.4961133357383686
Iteration 900: Current error = 0.4203629074403199
Final error: 0.3830201787885762


Iteration 0: Current error = 115.65518296731382
Iteration 100: Current error = 53.229703118496936
Iteration 200: Current error = 26.13443752617794
Iteration 300: Current error = 13.150153278276491
Iteration 400: Current error = 6.8560742982437
Iteration 500: Current error = 3.7818182844315937
Iteration 600: Current error = 2.272207888970038
Iteration 700: Current error = 1.5281591404987085
Iteration 800: Current error = 1.160522958724651
Iteration 900: Current error = 0.9785934475805679
Final error: 0.8890960198202208


Iteration 0: Current error = 272.58370466374305
Iteration 100: Current error = 119.129149910661
Iteration 200: Current error = 57.541248804771904
Iteration 300: Current error = 28.604013366975156
Iteration 400: Current error = 14.771741646637032
Iteration 500: Current error = 8.081959905747162
Iteration 600: Current error = 4.819498100333257
Iteration 700: Current error = 3.2193732071068544
Iteration 800: Current error = 2.4316307225194684
Iteration 900: Current error = 2.0428135103151446
Final error: 1.8519615520577155


Iteration 0: Current error = 521.4993072788118
Iteration 100: Current error = 215.01522146415377
Iteration 200: Current error = 101.93213516394974
Iteration 300: Current error = 49.96692242725941
Iteration 400: Current error = 25.52177867371136
Iteration 500: Current error = 13.829591748572808
Iteration 600: Current error = 8.170981544326029
Iteration 700: Current error = 5.4107667604779275
Iteration 800: Current error = 4.057006604645261
Iteration 900: Current error = 3.3908330772719477
Final error: 3.064665286685935


Iteration 0: Current error = 830.6782570888444
Iteration 100: Current error = 323.62636239306937
Iteration 200: Current error = 150.584083120518
Iteration 300: Current error = 72.78813362871136
Iteration 400: Current error = 36.76963899360221
Iteration 500: Current error = 19.728861942425358
Iteration 600: Current error = 11.544711242895922
Iteration 700: Current error = 7.573997783521017
Iteration 800: Current error = 5.634059160154597
Iteration 900: Current error = 4.682079980549199
Final error: 4.216976768914078


Generate 6 specimens only changing the arc angle: Long edges

In [8]:
# net.update_shape(q_new)
q0 = q.copy()*0.8
net = Network_custom.from_fd(vertices, edges, q0, fixed, paths = paths, dir = directions)
l1_initial = net.l1.copy()
l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 1000, "damping": .01, "correction_scalar": 1., "tol": 1e-6})
# net.initialize_shape_optimizer(function_type = 'standard',  method = 'L-BFGS-B',options ={"maxiter": 10000, "maxfun": 1500000, "maxls": 5000})
net.optimize_vertices()
reference_point = [0,0,0]                         # The network will be scaler relative to this point

l_scalar_vec = np.linspace(.65, net.l_scalar, 6)
for l_scalar in l_scalar_vec:

    net.scale_vertices(reference_point, l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically
    print(np.min(net.l1_scaled/net.l0))

    R, th = net.arc_param()
    xyz = net.arc_points(n = n_points)
    net.auto_flip_curves(n = n_points)        # Automatically flip the curvature of the edges. Directions will become 1, -1, 1, -1, ...
    net.flip_curves(n = n_points)                   # Flip all the edges.
    net.flip_curves([8, 10], n = n_points)
    net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 
    net.net_plot(color=True, plot_type='arcs', elables=True, vlabels = True)
    net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)
    net.add_running_start(running_start_bools, running_end_bools, L_running, n_points)
    file_name = model_name + f'_ls{l_scalar:.2f}'
    # print(f"q_scalar: {q_scalar}, l_scalar: {l_scalar}, max stress: {np.max(net.f)/TPU_nl['A']}")

    g_code = replace_brackets(start_gcode, temperature_settings)
    g_code += make_purge_line(printing_params, gen, point0, point1, comment='Purge line', ef_extra=2.5)
    g_code += make_purge_line(printing_params, gen, point2, point3, comment='Purge line', ef_extra=1.0)
    g_code += add_net_to_gcode(printing_params, gen, net)

    g_code += end_gcode
    # with open(os.path.join('DATA', 'generated_gcodes', file_name + '.gcode'), "w") as g_code_file:
    #     g_code_file.write(g_code)


Iteration 0: Current error = 115.65789172921023
Iteration 100: Current error = 53.229859052135936
Iteration 200: Current error = 26.134005715796928
Iteration 300: Current error = 13.149449627205293
Iteration 400: Current error = 6.8558381943274425
Iteration 500: Current error = 3.7816711339731
Iteration 600: Current error = 2.2722321315095915
Iteration 700: Current error = 1.5281672840806406
Iteration 800: Current error = 1.1605870922096484
Iteration 900: Current error = 0.9786192363189176
Final error: 0.8890910611255328
0.6447953142962868


0.7132430580187696


0.7816908017412525


0.8501385454637352


0.9185862891862181


0.987034032908701


Generate 6 specimens only changing the arc angle: Medium edges

In [9]:
# Note that the coordinates of the vertices will change after equilibrium is reached.
model_name = "validation_unit_s"
L_scale = 3.5
vertices    = np.array([[-10, 0, 0], [-5, 0,0], [0, 0, 0], [5, 0, 0], [10, 0, 0], [0,-10,0], [0,-5,0], [0,5,0], [0,10,0]])*L_scale  # The coordinates of the vertices in mm
vertices    = vertices@Rot(np.pi/4)
q_scalar = 1.4
sum_q0 = 0.02*q_scalar
sum_q1 = 0.02*q_scalar
sum_q2 = 0.01*q_scalar
offset = 1/32
ratio_0 = np.array([3/8-offset, 1/8, 1/8, 3/8+offset])
ratio_1 = np.array([3/8, 1/8-offset, 1/8+offset, 3/8])
ratio_2 = np.array([2/8-offset, 2/8+offset, 2/8, 2/8])

q = np.zeros(len(edges))
q[paths[0]] = sum_q0*ratio_0
q[paths[1]] = sum_q1*ratio_1
q[paths[2]] = sum_q2*ratio_2

q[connector_path]       *= 2.5
q[center_cross_path]    *= 2.5
q[paths[2]]             *= 2.5

q0 = q.copy()
net = Network_custom.from_fd(vertices, edges, q0, fixed, paths = paths, dir = directions)
l1_initial = net.l1.copy()
l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 1000, "damping": .01, "correction_scalar": 1., "tol": 1e-6})
# net.initialize_shape_optimizer(function_type = 'standard',  method = 'L-BFGS-B',options ={"maxiter": 10000, "maxfun": 1500000, "maxls": 5000})
net.optimize_vertices()
reference_point = [0,0,0]                         # The network will be scaler relative to this point

l_scalar_vec = np.linspace(.65, net.l_scalar, 6)
for l_scalar in l_scalar_vec:

    net.scale_vertices(reference_point, l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically
    print(np.min(net.l1_scaled/net.l0))

    R, th = net.arc_param()
    xyz = net.arc_points(n = n_points)
    net.auto_flip_curves(n = n_points)        # Automatically flip the curvature of the edges. Directions will become 1, -1, 1, -1, ...
    net.flip_curves(n = n_points)                   # Flip all the edges.
    net.flip_curves([8, 10], n = n_points)
    net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 

    net.net_plot(color=True, plot_type='arcs', elables=True, vlabels = True)
    
    net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)
    net.add_running_start(running_start_bools, running_end_bools, L_running, n_points)
    file_name = model_name + f'_ls{l_scalar:.2f}'
    # print(f"q_scalar: {q_scalar}, l_scalar: {l_scalar}, max stress: {np.max(net.f)/TPU_nl['A']}")

    g_code = replace_brackets(start_gcode, temperature_settings)
    g_code += make_purge_line(printing_params, gen, point0, point1, comment='Purge line', ef_extra=2.5)
    g_code += make_purge_line(printing_params, gen, point2, point3, comment='Purge line', ef_extra=1.0)
    g_code += add_net_to_gcode(printing_params, gen, net)

    g_code += end_gcode
    # with open(os.path.join('DATA', 'generated_gcodes', file_name + '.gcode'), "w") as g_code_file:
    #     g_code_file.write(g_code)

Iteration 0: Current error = 14.994507513105392
Iteration 100: Current error = 6.8447798571085405
Iteration 200: Current error = 3.3521489288082607
Iteration 300: Current error = 1.68337066883133
Iteration 400: Current error = 0.876109577366797
Iteration 500: Current error = 0.4823985679705466
Iteration 600: Current error = 0.2892779685790491
Iteration 700: Current error = 0.19417081348845128
Iteration 800: Current error = 0.14720169539306058
Iteration 900: Current error = 0.12396709388303823
Final error: 0.11254373693795418
0.6444496597202197


0.712792616176446


0.7811355726326723


0.8494785290888985


0.9178214855451249


0.9861644420013511


Generate 6 specimens only changing the arc angles: Short edges

In [10]:
model_name = "validation_unit_xs"
sum_q0 = 0.02*q_scalar
sum_q1 = 0.02*q_scalar
sum_q2 = 0.01*q_scalar
offset = 1/32
ratio_0 = np.array([3/8-offset, 1/8, 1/8, 3/8+offset])
ratio_1 = np.array([3/8, 1/8-offset, 1/8+offset, 3/8])
ratio_2 = np.array([2/8-offset, 2/8+offset, 2/8, 2/8])

q = np.zeros(len(edges))
q[paths[0]] = sum_q0*ratio_0
q[paths[1]] = sum_q1*ratio_1
q[paths[2]] = sum_q2*ratio_2

# scalar = 7
q[connector_path]       *= 2.5 * 0.65
q[center_cross_path]    *= 2.5 * 4
q[paths[2]]             *= 2.5 * 4

q0 = q.copy()
net = Network_custom.from_fd(vertices, edges, q0, fixed, paths = paths, dir = directions)
l1_initial = net.l1.copy()
l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 1000, "damping": .01, "correction_scalar": 1., "tol": 1e-6})
net.optimize_vertices()
reference_point = [0,0,0]                         # The network will be scaler relative to this point

l_scalar_vec = np.linspace(.65, net.l_scalar, 6)
for l_scalar in l_scalar_vec:

    net.scale_vertices(reference_point, l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically
    print(np.min(net.l1_scaled/net.l0))

    R, th = net.arc_param()
    xyz = net.arc_points(n = n_points)
    net.auto_flip_curves(n = n_points)        # Automatically flip the curvature of the edges. Directions will become 1, -1, 1, -1, ...
    net.flip_curves(n = n_points)                   # Flip all the edges.
    net.flip_curves([8, 10], n = n_points)
    net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 

    net.net_plot(color=True, plot_type='arcs', elables=True, vlabels = True)
    
    net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)
    net.add_running_start(running_start_bools, running_end_bools, L_running, n_points)
    file_name = model_name + f'_ls{l_scalar:.2f}'

    g_code = replace_brackets(start_gcode, temperature_settings)
    g_code += make_purge_line(printing_params, gen, point0, point1, comment='Purge line', ef_extra=2.5)
    g_code += make_purge_line(printing_params, gen, point2, point3, comment='Purge line', ef_extra=1.0)
    g_code += add_net_to_gcode(printing_params, gen, net)

    g_code += end_gcode
    # with open(os.path.join('DATA', 'generated_gcodes', file_name + '.gcode'), "w") as g_code_file:
    #     g_code_file.write(g_code)


Iteration 0: Current error = 56.50420945233983
Iteration 100: Current error = 18.73630843461566
Iteration 200: Current error = 8.730649025554358
Iteration 300: Current error = 4.1723522090663065
Iteration 400: Current error = 2.022377693452697
Iteration 500: Current error = 0.99167515638839
Iteration 600: Current error = 0.49211260529225437
Iteration 700: Current error = 0.24812840462266922
Iteration 800: Current error = 0.12833315293322944
Iteration 900: Current error = 0.06929218382706899
Final error: 0.04031773957238409
0.6472894949337872


0.714750407125353


0.7822113193169186


0.8496722315084841


0.91713314370005


0.9845940558916155
